# Med NLI

In [ ]:
import glob
import random
import os
import shutil

randseed = random.randint(1, 1000)
print("randseed: ", randseed)
rng = random.Random(randseed)

## make data

In [ ]:
def read_patient(path):
    file = open(path, "r")
    lines = file.readlines()
    
    documents = []
    document = []
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        if line=="":
            documents.append(" ".join(document))
            document = []
        else:
            document.append(line)            
    
    return documents

###################################
def create_samples(ptnums, 
                   category, 
                   dup = 2, 
                   limit_len = 300, random_samples=10, 
                   out_directory = "./data/", 
                   outfilename = "train.txt"):
    outtext = []
    
    rng.shuffle(ptnums)
    ptnums = ptnums[:random_samples]
    print("len(ptnums): ", len(ptnums))

    documents = []
    for p in range(len(ptnums)):
        documents.append(read_patient(ptnums[p]))
        
    
    if not os.path.exists(out_directory):
        os.makedirs(out_directory)
    
    if "train" in outfilename:
        file = open(out_directory+"/train_info.txt", "w")
        file.write(str(len(documents)))
        file.close()
        
    elif "test" in outfilename:
        file = open(out_directory+"/test_info.txt", "w")
        file.write(str(len(documents)))
        file.close()

    
    for t in range(len(ptnums)):
        print("ptnums[t]: ", ptnums[t])
        
        pat1 = t
        doc1 = rng.randint(0, len(documents[pat1])-1)
        doc2 = rng.randint(0, len(documents[pat1])-1)
        label = 0
        outtext.append(category+"\t"+str(pat1)+"\t"+str(pat1)+"\t"+
                   str(label)+"\t"+documents[pat1][doc1]+"\t"+documents[pat1][doc2])
        
        # ther other patient record 
        while(True):
            pat2 = random.randint(0, len(ptnums)-1)
            if pat1!=pat2:
                break
        
        doc1 = rng.randint(0, len(documents[pat1])-1)
        doc2 = rng.randint(0, len(documents[pat2])-1)
        label = 1
        outtext.append(category+"\t"+str(pat1)+"\t"+str(pat2)+"\t"+
                   str(label)+"\t"+documents[pat1][doc1]+"\t"+documents[pat2][doc2])
        
        
        # write
        if not os.path.exists(out_directory):
            os.makedirs(out_directory)
        file = open(out_directory+"/"+outfilename, "a")
        file.write("\n".join(outtext)+"\n")
        outtext = []
        file.close()
        

In [ ]:
path_input = "./../preprocessing/01_data4finetune"
train_paths = ["pts_SNUH_visit_2011to2020_heldout_train.txt"]
test_paths = ["pts_SNUH_visit_2011to2020_heldout_test.txt"]
categories = ["SNUH_visit_2011to2020"]

dup = 1
limit_len = 300

for t in range(len(train_paths)):
    print(path_input+"/"+train_paths[t])
    file = open(path_input+"/"+train_paths[t], "r")
    lines = file.readlines()

    train_pts = []
    for l in range(len(lines)):
        line = lines[l].strip("\n")        
        train_pts.append(path_input+"/"+line)
    file.close()
#     print(train_pts)

    
    test_pts = []
    file = open(path_input+"/"+test_paths[t], "r")
    lines = file.readlines()

    for l in range(len(lines)):
        line = lines[l].strip("\n")
        test_pts.append(path_input+"/"+line)
    file.close()
    #print(test_pts)

    # train dataset
    create_samples(ptnums=train_pts, category=categories[t], 
                   dup = dup, limit_len = limit_len,
                   random_samples = 4000, 
                   out_directory = "./data/",
                   outfilename = "train__yet.txt")

    # # test dataset
    create_samples(ptnums=test_pts, category=categories[t], 
                   dup = dup, limit_len = limit_len, 
                   random_samples = 200, 
                   out_directory = "./data/", 
                   outfilename = "test__yet.txt")

# Delete clues
- Delete text (Document type, Department, Date) which can make task too easy

In [ ]:
def del_stop_tokens(content):
    content = content.split(" ")
#     print("content: ", content)
    cut_s = 0
    for c in range(len(content)):
        if content[c]=="진료일":
            cut_s = c+2
            break
            
    content = content[cut_s:]
    content = " ".join(content)
    
    return content

In [ ]:
def make_del_text(in_path, out_path):
    file = open(in_path, "r")
    lines = file.readlines()
    file.close()

    new_lines = []
    for l in range(len(lines)):
        if (l+1)%1000==0:
            print(str(l)+" / "+str(len(lines)))
        
        line = lines[l].split("\t")
        if len(line)<4:
            continue

        try:
            cate = line[0]   # data category
            ptnum1 = line[1] # ptnum1
            ptnum2 = line[2] # ptnum2
            label = line[3]  # label
        
        except:
            print("line: ", line)

        content1 = del_stop_tokens(line[4].strip("\n"))
        content2 = del_stop_tokens(line[5].strip("\n"))

        new_lines.append(cate+"\t"+ptnum1+"\t"+ptnum2+"\t"+label+"\t"+content1+"\t"+content2)

    file = open(out_path, "w")
    file.write("\n".join(new_lines))
    file.close()
    print("Complete")


In [ ]:
make_del_text(in_path="./data/train__yet.txt", out_path="./data/train.txt")

In [ ]:
make_del_text(in_path="./data/test__yet.txt", out_path="./data/test.txt")